In [ ]:
!pip install unsloth

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.2/46.2 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 192.7/192.7 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 26.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.1/162.1 kB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 85.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 75.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 58.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from google.colab import userdata
userdata.get('huging_face_token')

In [ ]:
from unsloth import FastLanguageModel
import torch # Import PyTorch
from trl import SFTTrainer # Trainer for supervised fine-tuning (SFT)
from unsloth import is_bfloat16_supported # Checks if the hardware supports bfloat16 precision
# Hugging Face modules
from huggingface_hub import login # Lets you login to API
from transformers import TrainingArguments # Defines training hyperparameters
from datasets import load_dataset # Lets you load fine-tuning datasets
# Import weights and biases
import wandb
# Import kaggle secrets
# from kaggle_secrets import UserSecretsClient

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
Unsloth: Failed to patch Gemma3ForConditionalGeneration.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [2]:
# hugging_face_token = userdata.get('huging_face_token')
# wand_key = userdata.get('wandb_api_key')

In [1]:
# run = wandb.init(
#     project='Fine-tune-google-gemma-2b-it',
#     job_type="training",
#     anonymous="allow"
# )
# meta-llama/Llama-3.2-1B
# google/gemma-3-4b-it

In [3]:
# # Set parameters
# max_seq_length = 2048 # Define the maximum sequence length a model can handle (i.e. how many tokens can be processed at once)
# dtype = None # Set to default
# load_in_4bit = True # Enables 4 bit quantization — a memory saving optimization

# # Load the DeepSeek R1 model and tokenizer using unsloth — imported using: from unsloth import FastLanguageModel
# model, tokenizer = FastLanguageModel.from_pretrained(
#     model_name="google/gemma-3-4b-it",
#     max_seq_length=max_seq_length, # Ensure the model can process up to 2048 tokens at once
#     dtype=dtype, # Use the default data type (e.g., FP16 or BF16 depending on hardware support)
#     load_in_4bit=load_in_4bit, # Load the model in 4-bit quantization to save memory
#     token=hugging_face_token, # Use hugging face token
# )

Testing on Genearal Prompt

In [ ]:
prompt = "What is the capital of France?"

In [ ]:
input_ids = tokenizer(prompt, return_tensors="pt").input_ids.cuda()
generated_ids = model.generate(input_ids)

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


In [ ]:
generated_text = tokenizer.decode(generated_ids[0], skip_special_tokens=True)

In [ ]:
generated_text

'What is the capital of France?\n\nWhich planet is known as the “Red Planet”?\n\nWhat is the largest ocean on Earth?'

Testing on food ordering prompt

In [ ]:
sample_input = "Get me a  of Chicken Biryani along with two Mutton Combo."couple

In [ ]:
import json
with open('/content/manual_reviewed_menu_data.json', 'r') as josnfile:
  data = json.load(josnfile)

In [ ]:
def format_menu_for_prompt(menu_data):
    formatted_menu = []
    for category in menu_data:
        cat_name = category.get("category", "Uncategorized")
        for item in category.get("items", []):
            name = item.get("item_name", "Unknown Item")
            price = item.get("price", "N/A") or "N/A"
            formatted_menu.append(f"{name} ({cat_name}) - ₹{price}")
    return "\n".join(formatted_menu)


In [ ]:
formated_menu = format_menu_for_prompt(data)

In [ ]:
formated_menu

'Food Item (Breakfast) - ₹N/A\nFood Item 1 (Breakfast) - ₹N/A\nFood Item (Stew) - ₹N/A\nFood Item 1 (Stew) - ₹N/A\nFood tem (Dunks) - ₹N/A\nFood Iter 1 (Dunks) - ₹N/A\nFood Item 1 (Dunks) - ₹N/A\nChicken Biryani (Main course) - ₹100\nMutton Biryani (Main course) - ₹200\nMutton Combo (Main course) - ₹300'

In [ ]:
# Instruction-style prompt
chat_prompt = [
    {"role": "system", "content": "You are a helpful assistant that extracts orders from customer conversations."},
    {"role": "user", "content": f"Menu:\n{formated_menu}\n\nCustomer said: {sample_input}\n\nReturn JSON list with item_name, price, and quantity only."}
]

In [ ]:

# Step 1: Convert chat messages into a prompt string using chat template
chat_prompt_str = tokenizer.apply_chat_template(chat_prompt, tokenize=False)

# Step 2: Tokenize the prompt string and send to model's device
inputs = tokenizer(chat_prompt_str, return_tensors="pt").to(model.device)

# Generate
outputs = model.generate(
    **inputs,
    max_new_tokens=256,
    do_sample=False,
    temperature=0.1,
    top_p=1.0
)


In [ ]:
# Decode the response
response_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
print("🧠 Response:\n", response_text)

🧠 Response:
 user
You are a helpful assistant that extracts orders from customer conversations.

Menu:
Food Item (Breakfast) - ₹N/A
Food Item 1 (Breakfast) - ₹N/A
Food Item (Stew) - ₹N/A
Food Item 1 (Stew) - ₹N/A
Food tem (Dunks) - ₹N/A
Food Iter 1 (Dunks) - ₹N/A
Food Item 1 (Dunks) - ₹N/A
Chicken Biryani (Main course) - ₹100
Mutton Biryani (Main course) - ₹200
Mutton Combo (Main course) - ₹300

Customer said: Get me a couple of Chicken Biryani along with two Mutton Combo.

Return JSON list with item_name, price, and quantity only.
```json
[
  {
    "item_name": "Chicken Biryani",
    "price": 100,
    "quantity": 2
  },
  {
    "item_name": "Mutton Combo",
    "price": 300,
    "quantity": 2
  }
]
```


In [ ]:
import re, json
json_data = re.search(r'```json\n(.*?)\n```', response_text, re.DOTALL).group(1)
with open('output.json', 'w') as f:
    f.write(json_data)